In [1]:
using Pkg

In [2]:
# Pkg.activate("batterySC")

In [3]:
Pkg.status()

Project batterySC v0.1.0
Status `C:\Users\bourg\.julia\environments\batterySC\Project.toml`
  [336ed68f] CSV v0.10.7
  [a93c6f00] DataFrames v1.4.4
⌃ [2e9cd046] Gurobi v0.9.12
  [7073ff75] IJulia v1.23.3
⌃ [4076af6c] JuMP v0.21.10
⌅ [b8f27783] MathOptInterface v0.9.22
⌃ [c3e4b0f8] Pluto v0.19.9
  [37e2e46d] LinearAlgebra
Info Packages marked with ⌃ and ⌅ have new versions available, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [4]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames

In [13]:
Cdata = CSV.File("C:/Users/bourg/.julia/environments/batterySC/data/Julia_country.csv",header=1,delim=",") |> DataFrame;

In [35]:
#load the data and orgnize 
Cdata = CSV.File("C:/Users/bourg/.julia/environments/batterySC/data/Julia_country.csv",header=1,delim=",") |> DataFrame    #country data (Supply,Demand,Dsoc)
Combo = CSV.File("C:/Users/bourg/.julia/environments/batterySC/data/Julia_combo.csv",header=1,delim=",") |> DataFrame;   #Job country combinations 

In [36]:
Jobs_names = names(Combo)[2:size(Combo,2)]
com = Combo[1:size(Combo,1),2:size(Combo,2)]; #combination of job work 

In [37]:
country_names= Cdata[!,"Country"]
S= Cdata[!,"S"] #supply  nx1
D= Cdata[!,"D"] #Demand  nx1
Dsoc= Cdata[!,"Dsoc"]; #Social Demand nx1 

In [38]:
#parameters 
m= size(Jobs_names,1); # no. of jobs
n= size(S,1); #no of countries

In [44]:
model= Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "NonConvex", 2);

#variables 
@variable(model, 0 <= x[1:n, 1:m] <= 1) # whether to select the country? elements in x could only be 0 or 1
@variable(model, obj >= 0);

#Constraints 
@constraint(model, job_full[j in 1:m], sum(x[i, j] for i in 1:n) == 1) # each job is assigned to only 1 country
@constraint(model, country_job[i in 1:n, j in 1:m], x[i, j] <= com[i,j]) # only candidate countries can do the job
@constraint(model, obj*(sum(S'*x)) == (sum(D'*x)));

# Objective
@objective(model, Min, obj);

Academic license - for non-commercial use only - expires 2023-11-27


In [4]:
#optimize           
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 481 rows, 469 columns and 936 nonzeros
Model fingerprint: 0x5e9c2f1f
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [2e+09, 2e+12]
  QLMatrix range   [2e+10, 1e+13]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 468 rows and 375 columns

Continuous model is non-convex -- solving as a MIP.

Presolve removed 468 rows and 375 columns
Presolve time: 0.00s
Presolved: 386 rows, 188 columns, 930 nonzeros
Presolved model has 93 bilinear constraint(s)
Variable types: 188 continuous, 0 integer (0 binary)

Root relaxation: objective 6.844388e-02, 105 iterations, 0.00 seconds

    Nodes    |    Current Node

In [5]:
solution = JuMP.value.(x)

36×13 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮                        ⋮         
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0 

In [6]:
#Print the results
        
# solution = JuMP.value.(x) #??? 
println("\nObjective value): ", JuMP.objective_value(model)) 


Objective value): 1.244132261861857


In [7]:
function myfind(c)
    a = similar(c, Int)
    count = 1
    @inbounds for i in eachindex(c)
        a[count] = i
        count += (c[i] != zero(eltype(c)))
    end
    return resize!(a, count-1)
end

myfind (generic function with 1 method)

In [8]:
for i in 1:size(solution,2)
    print(Jobs_names[i])
    print(" :")
    println(country_names[myfind(solution[:,i])])
end

Mining_Li :String31["Chile"]
Mining_Co :String31["Canada"]
Mining_Ni :String31["Canada"]
Mining_Graphite :String31["Canada"]
Mining_Mn :String31["Cote dIvoire"]
Processing_Li2CO3_LiOH :String31["Canada"]
Processing_CoSO4_NiSO4 :String31["Russia"]
Processing_MnSO4 :String31["Netherlands"]
Production_Cathode :String31["Korea"]
Production_Anode :String31["Korea"]
Production_Separators :String31["Korea"]
Production_Electrolytes :String31["Korea"]
Production_Cell_Pack :String31["Hungary"]


In [30]:
just= Model(Gurobi.Optimizer)
set_optimizer_attribute(just, "NonConvex", 2)
#variables 
@variable(just, 0 <= x[1:n, 1:m] <= 1);
@variable(just, obj >=0);

#Constraints 
@constraint(just, job_full[j in 1:m], sum(x[i, j] for i in 1:n) == 1);
@constraint(just, country_job[i in 1:n, j in 1:m], x[i, j] <= com[i,j]);
@constraint(just, obj*(sum(D'*x)) == (sum(Dsoc'*x)))


@objective(just, Min, obj);

Academic license - for non-commercial use only - expires 2023-10-02


In [51]:
just= Model(Gurobi.Optimizer)
set_optimizer_attribute(just, "NonConvex", 2)
#variables 
n = 2;
m = 3;
@variable(just, 0 <= x[1:n, 1:m] <= 1);
@variable(just, obj >=0);

#Constraints 
@constraint(just, job_full[j in 1:m], sum(x[i, j] for i in 1:n) == 1);
@constraint(just, country_job[i in 1:n, j in 1:m], x[i, j] <= com[i,j]);
@constraint(just, obj*(sum(D'*x)) == (sum(Dsoc'*x)))

@objective(just, Min, obj);

println(just)

Academic license - for non-commercial use only - expires 2023-11-27
Min obj
Subject to
 job_full[1] : x[1,1] + x[2,1] == 1.0
 job_full[2] : x[1,2] + x[2,2] == 1.0
 job_full[3] : x[1,3] + x[2,3] == 1.0
 x[1,1] >= 0.0
 x[2,1] >= 0.0
 x[1,2] >= 0.0
 x[2,2] >= 0.0
 x[1,3] >= 0.0
 x[2,3] >= 0.0
 obj >= 0.0
 x[1,1] <= 1.0
 x[2,1] <= 1.0
 x[1,2] <= 1.0
 x[2,2] <= 1.0
 x[1,3] <= 1.0
 x[2,3] <= 1.0



In [58]:
ls[1]

2

In [59]:
model = Model()
@variable(model, x[1:3])
ls = [2, 5, 1]
@constraint(model, c[i=1:3], x[i] <= ls[i])

3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 c[1] : x[1] <= 2.0
 c[2] : x[2] <= 5.0
 c[3] : x[3] <= 1.0

In [31]:
#optimize           
JuMP.optimize!(just)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 481 rows, 469 columns and 936 nonzeros
Model fingerprint: 0xa8d5034e
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [2e+10, 1e+13]
  QLMatrix range   [8e+08, 7e+11]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 468 rows and 375 columns

Continuous model is non-convex -- solving as a MIP.

Presolve removed 468 rows and 375 columns
Presolve time: 0.00s
Presolved: 386 rows, 188 columns, 930 nonzeros
Presolved model has 93 bilinear constraint(s)
Variable types: 188 continuous, 0 integer (0 binary)

Root relaxation: objective 1.243159e-03, 95 iterations, 0.00 seconds

    Nodes    |    Current Node 

In [35]:
Jsolution = JuMP.value.(x);
for i in 1:size(Jsolution,2)
    print(Jobs_names[i])
    print(" :")
    println(country_names[myfind(Jsolution[:,i])])
end

Mining_Li :String31["Portugal"]
Mining_Co :String31["Canada"]
Mining_Ni :String31["Canada"]
Mining_Graphite :String31["Canada"]
Mining_Mn :String31["Malaysia"]
Processing_Li2CO3_LiOH :String31["Canada"]
Processing_CoSO4_NiSO4 :String31["United States"]
Processing_MnSO4 :String31["United States"]
Production_Cathode :String31["United States"]
Production_Anode :String31["Korea"]
Production_Separators :String31["Korea"]
Production_Electrolytes :String31["Korea"]
Production_Cell_Pack :String31["United States"]
